In [175]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

Load the TSV file containing information about titles:

In [124]:
ratings_dtype = {
    'tconst': str,
    'averageRating': float,
    'numVotes': int
}

ratings = pd.read_csv('./title.ratings.tsv.gz',
                      sep='\t', header=0, dtype=ratings_dtype)

In [125]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1383
1,tt0000002,6.5,162
2,tt0000003,6.6,971
3,tt0000004,6.4,98
4,tt0000005,6.2,1663


Load TSV file containing IMDb rating and votes information for titles:

In [190]:
title_dtype = {
    'tconst': str,
    'titleType': str,
    'primaryTitle': str,
    'originalTitle': str,
    'isAdult': int,
    'startYear': str,
    'endYear': str,
    'runtimeMinutes': object,
    'genres': str
}

title = pd.read_csv('./title.basics.tsv.gz',
                    sep='\t', header=0, dtype=title_dtype)

In [191]:
title.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


What are the types of titles?

In [192]:
title['titleType'].unique()

array(['short', 'movie', 'tvMovie', 'tvSeries', 'tvEpisode', 'tvShort',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame'], dtype=object)

Create variable containing TV series:

In [193]:
series = title[title['titleType'] == 'tvSeries']

In [194]:
series.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34995,tt0035599,tvSeries,Voice of Firestone Televues,Voice of Firestone Televues,0,1943,1947,15,\N
37626,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0,1946,1950,15,\N
38460,tt0039120,tvSeries,Americana,Americana,0,1947,1949,30,"Family,Game-Show"
38461,tt0039121,tvSeries,Birthday Party,Birthday Party,0,1947,1949,30,Family
38462,tt0039122,tvSeries,The Borden Show,The Borden Show,0,1947,\N,30,"Comedy,Music"


Get all the unique genres the TV series belong to:

In [195]:
genres_with_dupes = series['genres'].map(lambda x: x.split(','))
genres_with_dupes_list = [y for x in genres_with_dupes for y in x if y != '\\N']
genres = list(set(genres_with_dupes_list))
genres

['Adult',
 'Horror',
 'Family',
 'News',
 'Short',
 'Drama',
 'Talk-Show',
 'War',
 'Musical',
 'Sport',
 'History',
 'Music',
 'Crime',
 'Sci-Fi',
 'Documentary',
 'Biography',
 'Western',
 'Fantasy',
 'Comedy',
 'Thriller',
 'Reality-TV',
 'Game-Show',
 'Mystery',
 'Adventure',
 'Animation',
 'Action',
 'Romance']

Set genres column in series dataframe to array of genres:

In [196]:
# To get rid of SettingWithCopyWarning. See https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
series.is_copy = False

series['genres'] = genres_with_dupes
series.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34995,tt0035599,tvSeries,Voice of Firestone Televues,Voice of Firestone Televues,0,1943,1947,15,[\N]
37626,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0,1946,1950,15,[\N]
38460,tt0039120,tvSeries,Americana,Americana,0,1947,1949,30,"[Family, Game-Show]"
38461,tt0039121,tvSeries,Birthday Party,Birthday Party,0,1947,1949,30,[Family]
38462,tt0039122,tvSeries,The Borden Show,The Borden Show,0,1947,\N,30,"[Comedy, Music]"


We want counts of genres per start year, so let's create a data structure with a form like this:
```
{
  '1947': {
    'Action': 92,
    'Adult': 11,
    'Adventure': 91
  }
}
```

In [197]:
start_years = [i for i in title['startYear'].unique() if i != '\\N']
title['startYear'].unique()

array(['1894', '1892', '1893', '1895', '1896', '1898', '1897', '1900',
       '1899', '1901', '1904', '1903', '1902', '1905', '1912', '1907',
       '1906', '1908', '1910', '1909', '1914', '1911', '1913', '1915',
       '1919', '1916', '1917', '1918', '1936', '1925', '1920', '1922',
       '1921', '1923', '1924', '1928', '1926', '1927', '1929', '1993',
       '1935', '1930', '1942', '1933', '1934', '1931', '1939', '1932',
       '1950', '1938', '1951', '1937', '1945', '1983', '1946', '1940',
       '1944', '1949', '1941', '1952', '1970', '1957', '1943', '1948',
       '1959', '2001', '1947', '1953', '1954', '1965', '1973', '1961',
       '1995', '1955', '1962', '1958', '1956', '1964', '1960', '1967',
       '1968', '1963', '1972', '1969', '1985', '1971', '1966', '1976',
       '1986', '1990', '1977', '1979', '1978', '1981', '1988', '1975',
       '1974', '1989', '2016', '\\N', '1980', '2010', '2018', '1984',
       '1982', '1991', '1987', '1992', '1994', '1999', '2005', '1998',
       

In [198]:
def count_genres(year):
  matches = series.loc[series['startYear'] == year]
  gs = [y for x in matches['genres'] for y in x if y != '\\N']
  return {x: gs.count(x) for x in gs}

genre_counts = []
for yr in start_years:
  gc = count_genres(yr)
  if bool(gc): genre_counts.append({yr: gc})

In [205]:
genre_counts[1:5]

[{'1910': {'News': 1}},
 {'1916': {'Talk-Show': 1}},
 {'1936': {'Comedy': 3, 'Family': 1, 'Music': 3, 'Sport': 1, 'Talk-Show': 1}},
 {'1922': {'News': 1}}]

In [204]:
genre_ratios = pd.Series(genres_with_dupes_list).value_counts()
genre_ratios.head()

Comedy         36302
Drama          23299
Documentary    16715
Reality-TV     13859
Talk-Show      11077
dtype: int64

Plot of genre ratios across all TV series ever made:

In [ ]:
# plot()

### What are the average ratings for genres over the years?
First, let's add ratings information to TV series:

In [208]:
series_with_ratings = pd.merge(series, ratings, how='outer', on=['tconst'])
series_with_ratings.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0035599,tvSeries,Voice of Firestone Televues,Voice of Firestone Televues,0.0,1943,1947,15,[\N],NaN,NaN
1,tt0038276,tvSeries,You Are an Artist,You Are an Artist,0.0,1946,1950,15,[\N],NaN,NaN
2,tt0039120,tvSeries,Americana,Americana,0.0,1947,1949,30,"[Family, Game-Show]",3.7,6.0
3,tt0039121,tvSeries,Birthday Party,Birthday Party,0.0,1947,1949,30,[Family],NaN,NaN
4,tt0039122,tvSeries,The Borden Show,The Borden Show,0.0,1947,\N,30,"[Comedy, Music]",NaN,NaN
